In [ ]:
!pip install nba_api

import pandas as pd
from nba_api.stats.static import players
from nba_api.stats.endpoints import playergamelog, shotchartdetail
import time

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.3/285.3 kB 3.9 MB/s eta 0:00:00


In [ ]:
# Get Stephen Curry's ID
steph = [p for p in players.get_players() if p['full_name'] == "Stephen Curry"][0]
steph_id = steph['id']

# Seasons from 2009-10 to 2023-24
seasons = [f"{y}-{str(y+1)[-2:]}" for y in range(2009, 2024)]

In [ ]:
# Game logs
all_game_logs = []
for season in seasons:
    gamelog = playergamelog.PlayerGameLog(player_id=steph_id, season=season, season_type_all_star='Regular Season')
    df = gamelog.get_data_frames()[0]
    df['SEASON'] = season
    all_game_logs.append(df)
    time.sleep(0.6)

games = pd.concat(all_game_logs, ignore_index=True)
games['GAME_DATE'] = pd.to_datetime(games['GAME_DATE'])
games['DAY_OF_WEEK'] = games['GAME_DATE'].dt.day_name()
games.columns = games.columns.str.upper()
games = games[['GAME_ID', 'SEASON', 'GAME_DATE', 'DAY_OF_WEEK', 'PTS']]

In [ ]:
# Shot charts
all_shots = []
for season in seasons:
    shot_data = shotchartdetail.ShotChartDetail(
        team_id=0,
        player_id=steph_id,
        season_nullable=season,
        season_type_all_star='Regular Season',
        context_measure_simple='FGA'
    )
    df = shot_data.get_data_frames()[0]
    df['SEASON'] = season
    all_shots.append(df)
    time.sleep(0.6)

shots = pd.concat(all_shots, ignore_index=True)

In [ ]:
# Merge on GAME_ID and SEASON
combined = shots.merge(games, on=['GAME_ID', 'SEASON'], how='left', suffixes=('', '_GAMES'))


print("shots columns:", shots.columns)
print("games columns:", games.columns)


# Clean columns for Tableau
final = combined[[
    'GAME_ID', 'SEASON', 'GAME_DATE_GAMES', 'DAY_OF_WEEK', 'PTS',
    'LOC_X', 'LOC_Y', 'SHOT_DISTANCE', 'SHOT_MADE_FLAG',
    'ACTION_TYPE', 'SHOT_TYPE', 'SHOT_ZONE_BASIC',
    'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE'
]].rename(columns={'GAME_DATE_GAMES': 'GAME_DATE'})

shots columns: Index(['GRID_TYPE', 'GAME_ID', 'GAME_EVENT_ID', 'PLAYER_ID', 'PLAYER_NAME',
       'TEAM_ID', 'TEAM_NAME', 'PERIOD', 'MINUTES_REMAINING',
       'SECONDS_REMAINING', 'EVENT_TYPE', 'ACTION_TYPE', 'SHOT_TYPE',
       'SHOT_ZONE_BASIC', 'SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE', 'SHOT_DISTANCE',
       'LOC_X', 'LOC_Y', 'SHOT_ATTEMPTED_FLAG', 'SHOT_MADE_FLAG', 'GAME_DATE',
       'HTM', 'VTM', 'SEASON'],
      dtype='object')
games columns: Index(['GAME_ID', 'SEASON', 'GAME_DATE', 'DAY_OF_WEEK', 'PTS'], dtype='object')


In [ ]:
final

,GAME_ID,SEASON,GAME_DATE,DAY_OF_WEEK,PTS,LOC_X,LOC_Y,SHOT_DISTANCE,SHOT_MADE_FLAG,ACTION_TYPE,SHOT_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE
0,0020900015,2009-10,2009-10-28,Wednesday,14,99,249,26,0,Jump Shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.
1,0020900015,2009-10,2009-10-28,Wednesday,14,-122,145,18,1,Step Back Jump shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.
2,0020900015,2009-10,2009-10-28,Wednesday,14,-60,129,14,0,Jump Shot,2PT Field Goal,In The Paint (Non-RA),Center(C),8-16 ft.
3,0020900015,2009-10,2009-10-28,Wednesday,14,-172,82,19,0,Jump Shot,2PT Field Goal,Mid-Range,Left Side(L),16-24 ft.
4,0020900015,2009-10,2009-10-28,Wednesday,14,-68,148,16,0,Jump Shot,2PT Field Goal,Mid-Range,Left Side Center(LC),16-24 ft.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17090,0022301222,2023-24,2023-12-08,Friday,34,129,243,27,1,Step Back Jump shot,3PT Field Goal,Above the Break 3,Right Side Center(RC),24+ ft.
17091,0022301222,2023-24,2023-12-08,Friday,34,-4,290,29,0,Pullup Jump shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.
17092,0022301222,2023-24,2023-12-08,Friday,34,1,294,29,1,Jump Shot,3PT Field Goal,Above the Break 3,Center(C),24+ ft.
17093,0022301222,2023-24,2023-12-08,Friday,34,70,142,15,1,Driving Floating Jump Shot,2PT Field Goal,Mid-Range,Center(C),8-16 ft.


In [ ]:
# Export CSV
final.to_csv('steph_curry_shot_chart_combined.csv', index=False)